In [11]:
import os
import csv
import json
import requests
import time
import jwt
from pandas.io.json import json_normalize

# Read data from CSV file
csv_file = '/Users/toms/code/s37-snowflake/files/accounts_plain.csv'
json_file = '/Users/toms/code/s37-snowflake/files/accounts_plain.json'

data = []

with open(csv_file, 'r') as csvf:
    csv_reader = csv.DictReader(csvf)
    for row in csv_reader:
        row['accountid'] = int(row['accountid'])
        data.append(row)

vault_req = []
for i in range(len(data)):
    vault_req.append({'fields': data[i]})

# Write data to JSON file
with open(json_file, 'w') as jsonf:
    json.dump(vault_req, jsonf, indent=4)

print(f'CSV file "{csv_file}" has been converted to JSON file "{json_file}".')

CSV file "/Users/toms/code/s37-snowflake/files/accounts_plain.csv" has been converted to JSON file "/Users/toms/code/s37-snowflake/files/accounts_plain.json".


In [16]:
# Open the file in read mode
file_path = '/Users/toms/code/s37-snowflake/files/accounts_plain.json'

with open(file_path, 'r') as file:
    # Load the JSON data
    json_data = json.load(file)

clients_vault_dict = {'quorum': False, 'records': json_data, 'tokenization': True}
clients_vault_json = json.dumps(clients_vault_dict, indent=4, sort_keys=True)

# print(clients_vault_json)

In [18]:
def getSignedJWT(credsFile):
   # credsFile is the filepath to your credentials.json file
   # Load the credentials.json file into an object called creds
   fd = open(credsFile)
   creds = json.load(fd)
   fd.close()
   
   # Create the claims object with the data in the creds object
   claims = {
       "iss": creds["clientID"],
       "key": creds["keyID"], 
       "aud": creds["tokenURI"], 
       "exp": int(time.time()) + (3600), # JWT expires in Now + 60 minutes
       "sub": creds["clientID"], 
   }
   # Sign the claims object with the private key contained in the creds object
   signedJWT = jwt.encode(claims, creds["privateKey"], algorithm='RS256') 
   return signedJWT, creds  

def getBearerToken(signedJWT, creds):
   # Request body parameters
   body = {
       'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer',
       'assertion': signedJWT,
   }
   # Request URI (== https://api.skylfow.dev/v1/auth/sa/oauth/token) 
   tokenURI = creds["tokenURI"]
   
   # Send the POST request using your favorite Python HTTP request lib
   r = requests.post(tokenURI, json=body)
   return r.text

jwtToken, creds = getSignedJWT('/Users/toms/code/s37-snowflake/credentials/admin-sa.json')
bearerToken = json.loads(getBearerToken(jwtToken, creds))
token = ((bearerToken['accessToken']))
header = "Bearer "+ token

url = "https://ebfc9bee4242.vault.skyflowapis.com/v1/vaults/vc31365cea49494eab417dbc439589f8/account"
headers={}
headers["Content-Type"] = "application/json"
headers["Authorization"] = header

response = (requests.request("POST", url, headers=headers, data=clients_vault_json)).json()
print(response)
#print(json.dumps(response['records'][0], indent=4, sort_keys=True))

{'records': [{'skyflow_id': '738625c4-8225-446e-8664-a7a59932e2e2', 'tokens': {'accountid': '8a251c65-5eac-498e-bc33-7df76a5ba96d', 'birthdate': '8785-06-02', 'email': 'xvzfgiqlrimtrestjboi@zhmirbxhad.com', 'firstname': '39e12c9c-98c9-4fe0-96f5-0dbfed3a7f99', 'gender': '9ae2e93f-acf5-4c0c-835c-6bdfdd90ac48', 'lastname': 'c9935e97-db0b-4795-9794-20dddad06300', 'middleinitial': 'ec9b5f29-22fc-4bf4-b652-6c60f8e7b251', 'mobilenumber': '1878b0b6-4377-4319-8517-7ece6cc3404a', 'statecode': 'WJ', 'suffix': '86880578-0604-4f76-9dbc-9fd72b99252e', 'username': '4e82232a-8c71-47b7-ba07-586c1e7055e0'}}, {'skyflow_id': 'fa05e4f4-46b4-4de8-87d1-9acf556c598f', 'tokens': {'accountid': 'aa4bd842-c076-4ea5-a976-6be9dd7c2952', 'birthdate': '3259-07-04', 'email': 'aozayntqclmnejjrqeds@rjzlnvdfeq.com', 'firstname': 'f2b799f5-2977-4a88-879f-d58a58efa813', 'gender': '92774734-8df6-40a0-a97a-70fa773a533c', 'lastname': 'c7ac7594-3a4b-40af-a0db-a8daee0bc078', 'middleinitial': 'fea01a6a-fb70-4aee-bed7-04708793a6c

In [19]:
print(response['records'])
print(clients_vault_dict)
print(json_data)
for i in range(len(response['records'])):
    json_data[i]['accountid'] = response['records'][i]['tokens']['accountid']
    json_data[i]['firstname'] = response['records'][i]['tokens']['firstname']
    json_data[i]['lastname'] = response['records'][i]['tokens']['lastname']
    json_data[i]['email'] = response['records'][i]['tokens']['email']
    json_data[i]['gender'] = response['records'][i]['tokens']['gender']
    json_data[i]['suffix'] = response['records'][i]['tokens']['suffix']
    json_data[i]['mobilenumber'] = response['records'][i]['tokens']['mobilenumber']
    json_data[i]['statecode'] = response['records'][i]['tokens']['statecode']
    json_data[i]['username'] = response['records'][i]['tokens']['username']
    json_data[i]['middleinitial'] = response['records'][i]['tokens']['middleinitial']
    json_data[i]['birthdate'] = response['records'][i]['tokens']['birthdate']

    del json_data[i]['fields']
print(json_data)
df = json_normalize(json_data)
print(df)

[{'skyflow_id': '738625c4-8225-446e-8664-a7a59932e2e2', 'tokens': {'accountid': '8a251c65-5eac-498e-bc33-7df76a5ba96d', 'birthdate': '8785-06-02', 'email': 'xvzfgiqlrimtrestjboi@zhmirbxhad.com', 'firstname': '39e12c9c-98c9-4fe0-96f5-0dbfed3a7f99', 'gender': '9ae2e93f-acf5-4c0c-835c-6bdfdd90ac48', 'lastname': 'c9935e97-db0b-4795-9794-20dddad06300', 'middleinitial': 'ec9b5f29-22fc-4bf4-b652-6c60f8e7b251', 'mobilenumber': '1878b0b6-4377-4319-8517-7ece6cc3404a', 'statecode': 'WJ', 'suffix': '86880578-0604-4f76-9dbc-9fd72b99252e', 'username': '4e82232a-8c71-47b7-ba07-586c1e7055e0'}}, {'skyflow_id': 'fa05e4f4-46b4-4de8-87d1-9acf556c598f', 'tokens': {'accountid': 'aa4bd842-c076-4ea5-a976-6be9dd7c2952', 'birthdate': '3259-07-04', 'email': 'aozayntqclmnejjrqeds@rjzlnvdfeq.com', 'firstname': 'f2b799f5-2977-4a88-879f-d58a58efa813', 'gender': '92774734-8df6-40a0-a97a-70fa773a533c', 'lastname': 'c7ac7594-3a4b-40af-a0db-a8daee0bc078', 'middleinitial': 'fea01a6a-fb70-4aee-bed7-04708793a6c4', 'mobilen

/var/folders/m6/8t6n7ftd6_3fsykmqwxtp9hm0000gn/T/ipykernel_93126/4100493181.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(json_data)


In [21]:
import pandas as pd
from google.cloud import storage

BUCKET_NAME = 'clients_tokenized'
file_path = '/Users/toms/code/s37-snowflake/files/'
file_name = 'account_tokenized.csv'

def get_storage_client():
    storage_client = storage.Client.from_service_account_json("/Users/toms/code/recurve-demo/genial-shuttle-365622-20499cb6a807.json")
    return storage_client

def save_df_as_csv(df, file_path):
    """Saves a Pandas DataFrame as a CSV file."""
    df.to_csv(file_path, index=False)
    print(f'DataFrame saved as CSV to {file_path}.')

def upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to the given Google Cloud Storage bucket."""
    storage_client = get_storage_client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)
    print(f'File {source_file_path} uploaded to {destination_blob_name} in {bucket_name} bucket.')

print(df)
save_df_as_csv(df, file_path+file_name)
# upload_file_to_gcs(BUCKET_NAME, file_path+file_name, "clients_tokenized.csv")


                               accountid  \
0   8a251c65-5eac-498e-bc33-7df76a5ba96d   
1   aa4bd842-c076-4ea5-a976-6be9dd7c2952   
2   a9267399-4be7-4e8a-b740-cdb8b42a5785   
3   361835d7-9979-4154-b44c-8f46531e682c   
4   cdef4449-50eb-4eb6-a93f-4ec56ec7a4e4   
5   d49dd59f-19b6-4083-a732-e0b6e3d8137f   
6   bf00a8ea-f95d-47d5-b2ba-0e3b2c4e5132   
7   64c462e2-9f49-40d2-9c08-0824a236dc71   
8   31a2d5a4-5ae8-4d93-a048-586c77145a93   
9   33445445-0c12-478f-ac2a-a44dc49aaf1f   
10  1fc373bc-8f62-490e-a38f-cbed80de7f49   
11  ba4208a7-b870-490d-b09d-a6af58d9de63   
12  17bf17cd-2c56-44e9-9185-19cb5f7fc6bb   
13  5b9652dd-ffef-4960-9480-a7d44ae16b58   
14  907dd144-e58f-40cc-9f34-f9b77c28c57f   
15  1612c062-1c1c-4f19-81aa-25e0c0453ebf   
16  bd39dbc0-6e13-4a39-8502-69998344b81b   
17  9be4111e-82b2-4e4d-b192-2cd15ef19e1e   
18  febc4b2d-ecb5-487c-bce4-93d575b14582   
19  5083f036-b25e-4741-aa8b-89176c3bc25c   
20  b7e4165f-2fa9-4f5c-9082-64dc94bc45c2   
21  97f8c289-1a3f-49cf-89dd-78fa